In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("EXGEUS.csv")
tsValues = data['EXGEUS']
import math
tsValuesLog = [math.log(ele) for ele in tsValues]

In [2]:
PredictionOutputMAE = pd.DataFrame(index=['monthly','3_month','6_month','12_month'])
PredictionOutputRMSE = pd.DataFrame(index=['monthly','3_month','6_month','12_month'])
startPrediction = 100
totalData = len(tsValuesLog)

In [3]:
totalData

157

In [4]:
def mean_error(pred,real):
    return np.mean([pred[i]-real[i] for i in range(len(pred))])
def mean_absolute_error(pred,real):
    return np.mean([np.abs(pred[i]-real[i]) for i in range(len(pred))])
def root_mean_square_error(pred,real):
    return np.sqrt(np.mean([pow(pred[i]-real[i],2) for i in range(len(pred))]))

In [6]:
from statsmodels.tsa.arima_model import ARMA
mae_ = []
rmse_ = []
p=1
q=1
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        try:
            model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
            result = model.fit()
            pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
        except ValueError:
            pred.append(tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_Diff_'+str(p)+'_'+str(q)] = rmse_

In [8]:
from statsmodels.tsa.arima_model import ARMA
mae_ = []
rmse_ = []
test_queue = [[1,1],[0,1],[1,0]]
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        single_pred = []
        for [p,q] in test_queue:
            try:
                model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
                result = model.fit()
                single_pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
            except ValueError:
                single_pred.append(tsValuesLog[i+startPrediction-1])
        pred.append(np.mean(single_pred))
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_Mean'] = mae_
PredictionOutputRMSE['AR_Diff_Mean'] = rmse_

C:\Users\v-weiyzh\AppData\Roaming\Python\Python36\site-packages\scipy\signal\signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
C:\Users\v-weiyzh\AppData\Roaming\Python\Python36\site-packages\scipy\signal\signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
C:\Users\v-weiyzh\AppData\Roaming\Python\Python36\site-packages\scipy\signal\signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead o

In [9]:
PredictionOutputRMSE

,AR_Diff_1_1,AR_Diff_Mean
monthly,0.020179,0.020275
3_month,0.043734,0.043806
6_month,0.066201,0.066084
12_month,0.107126,0.106882


In [7]:
#benchmark
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        pred.append(tsValuesLog[startPrediction-1+i])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
PredictionOutputMAE['RandomWalk'] = mae_
PredictionOutputRMSE['RandomWalk'] = rmse_

In [8]:
PredictionOutputMAE

,RandomWalk
monthly,0.017478
3_month,0.034895
6_month,0.049859
12_month,0.083989


In [9]:
#test performance random walk with drift
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        monthly_change = [tsValuesLog[j]-tsValuesLog[j-l] for j in range(i+l,i+startPrediction)]
        mean_monthly_change = np.mean(monthly_change)
        pred.append(tsValuesLog[startPrediction+i-1]+mean_monthly_change)
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['RandomWalkWithDrift'] = mae_
PredictionOutputRMSE['RandomWalkWithDrift'] = rmse_

In [10]:
PredictionOutputMAE

,RandomWalk,RandomWalkWithDrift
monthly,0.017478,0.017702
3_month,0.034895,0.036400
6_month,0.049859,0.055378
12_month,0.083989,0.096389


In [11]:
%%time
from statsmodels.tsa.arima_model import ARMA
#out-of-sample-test
mae_ = []
rmse_ = []
p=1
q=1
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        try:
            model = ARMA(tsValuesLog[i:i+startPrediction],order=(p,q))
            result = model.fit()
            pred.append(result.predict(start=startPrediction,end=startPrediction+l-1)[l-1])
        except ValueError:
            pred.append(tsValuesLog[startPrediction+i-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_'+str(p)+'_'+str(q)] = rmse_

Wall time: 20.2 s


In [12]:
%%time
mae_ = []
rmse_ = []
p=2
q=0
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        try:
            model = ARMA(tsValuesLog[i:i+startPrediction],order=(p,q))
            result = model.fit()
            pred.append(result.predict(start=startPrediction,end=startPrediction+l-1)[l-1])
        except ValueError:
            pred.append(tsValuesLog[startPrediction+i-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_'+str(p)+'_'+str(q)] = rmse_

Wall time: 15.5 s


In [13]:
PredictionOutputMAE

,RandomWalk,RandomWalkWithDrift,AR_1_1,AR_2_0
monthly,0.017478,0.017702,0.016133,0.016702
3_month,0.034895,0.036400,0.035403,0.038150
6_month,0.049859,0.055378,0.053430,0.059155
12_month,0.083989,0.096389,0.074341,0.080379


In [5]:
tsValuesLog_d_1 = [0]
tsValuesLog_d_1 += [tsValuesLog[i+1]-tsValuesLog[i] for i in range(totalData-1)]

In [15]:
mae_ = []
rmse_ = []
p=1
q=0
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_Diff_'+str(p)+'_'+str(q)] = rmse_

In [16]:
%%time
mae_ = []
rmse_ = []
p=0
q=1
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_Diff_'+str(p)+'_'+str(q)] = rmse_

Wall time: 11.4 s


In [25]:
#Unobserved model
import statsmodels.api as sm

class UnobservedComponentModel:
    def __init__(self):
        self.model_name = "UnobservedComponentModel"
        return

    def fit(self, ts):
        unrestricted_model = {
            'level': 'local linear trend', 'cycle': False , 'seasonal': 12
        }

        model = sm.tsa.UnobservedComponents(endog=ts, **unrestricted_model)
        self.trained_model = model.fit()
        return self

    def predict(self, next_n_prediction):
        prediction = self.trained_model.forecast(steps=next_n_prediction)
        return prediction

In [26]:
%%time
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-(l-1)):
        model = UnobservedComponentModel()
        model.fit(tsValuesLog[i:i+startPrediction])
        pred.append(model.predict(l)[l-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))

C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check

Wall time: 1min 30s


In [23]:
PredictionOutputMAE["UnobservedComponentModel"] = mae_
PredictionOutputRMSE["UnobservedComponentModel"] = rmse_

In [27]:
PredictionOutputMAE["UnobservedComponentModel_With_Season"] = mae_
PredictionOutputRMSE["UnobservedComponentModel_With_Season"] = rmse_

In [28]:
PredictionOutputMAE

,RandomWalk,RandomWalkWithDrift,AR_1_1,AR_2_0,AR_Diff_1_0,AR_Diff_0_1,UnobservedComponentModel,UnobservedComponentModel_With_Season
monthly,0.017478,0.017702,0.016133,0.016702,0.017047,0.016351,0.017704,0.017781
3_month,0.034895,0.036400,0.035403,0.038150,0.037099,0.035971,0.038160,0.033980
6_month,0.049859,0.055378,0.053430,0.059155,0.053068,0.052811,0.057252,0.057595
12_month,0.083989,0.096389,0.074341,0.080379,0.092079,0.091961,0.107377,0.107884


In [29]:
PredictionOutputRMSE

,RandomWalk,RandomWalkWithDrift,AR_1_1,AR_2_0,AR_Diff_1_0,AR_Diff_0_1,UnobservedComponentModel,UnobservedComponentModel_With_Season
monthly,0.021060,0.021374,0.020067,0.020974,0.020787,0.020112,0.021403,0.021345
3_month,0.042205,0.043806,0.043698,0.046264,0.044232,0.043660,0.045260,0.041430
6_month,0.062737,0.067512,0.063129,0.068491,0.066349,0.065876,0.069860,0.065950
12_month,0.096899,0.111920,0.087163,0.091769,0.106933,0.106712,0.119083,0.118693


In [30]:
PredictionOutputMAE.to_csv("EXGEUS_MAE.csv")

In [31]:
PredictionOutputRMSE.to_csv("EXGEUS_RMAE.csv")